In [31]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import LeagueGameLog
import pandas as pd

# --- Config ---
season_variable        = '2021-22'
games_threshold        = 50
minutes_threshold      = 15

# --- Grab every player, not just active ones ---
all_players       = players.get_players()
player_id_lookup  = {p['id']: p for p in all_players}

# --- Pull game logs ---
log = LeagueGameLog(
    season=season_variable,
    season_type_all_star='Regular Season',
    player_or_team_abbreviation='P'
)
log_df = log.get_data_frames()[0]
log_df['MIN'] = pd.to_numeric(log_df['MIN'], errors='coerce')

# --- Summarize ---
summary_df = (
    log_df
    .groupby('PLAYER_ID')
    .agg(GP_TOTAL=('GAME_ID','count'),
         MIN_TOTAL=('MIN','sum'))
    .reset_index()
)
summary_df['MIN_PG'] = summary_df['MIN_TOTAL'] / summary_df['GP_TOTAL']

# --- Filter by your thresholds ---
qualified_df = summary_df[
    (summary_df['GP_TOTAL'] >= games_threshold) &
    (summary_df['MIN_PG']  >= minutes_threshold)
]

print("Qualified IDs:", qualified_df['PLAYER_ID'].tolist())

# --- Build your final profiles ---
player_dic = []
for _, row in qualified_df.iterrows():
    pid = int(row['PLAYER_ID'])
    if pid in player_id_lookup:
        prof = player_id_lookup[pid].copy()
        prof['GP_TOTAL'] = int(row['GP_TOTAL'])
        prof['MIN_PG']   = round(float(row['MIN_PG']), 2)
        player_dic.append(prof)
    else:
        # This should no longer happen, since we used all_players
        print(f"⚠️ Couldn't find static entry for ID {pid}")

print([f"{p['full_name']} (ID {p['id']}): GP={p['GP_TOTAL']}, MIN/PG={p['MIN_PG']}"
       for p in player_dic])


Qualified IDs: [2544, 2546, 2730, 101108, 200752, 200768, 200782, 201142, 201143, 201144, 201145, 201152, 201566, 201567, 201568, 201569, 201571, 201580, 201586, 201587, 201588, 201933, 201935, 201939, 201942, 201949, 201950, 201959, 201976, 201980, 201988, 202066, 202339, 202340, 202355, 202357, 202397, 202684, 202685, 202692, 202694, 202696, 202699, 202704, 202709, 202710, 202711, 203082, 203083, 203084, 203085, 203087, 203095, 203107, 203109, 203114, 203115, 203200, 203210, 203463, 203468, 203471, 203473, 203484, 203486, 203490, 203493, 203496, 203497, 203500, 203503, 203507, 203526, 203552, 203897, 203903, 203914, 203915, 203920, 203926, 203932, 203935, 203937, 203939, 203944, 203952, 203954, 203991, 203992, 203994, 203999, 204001, 1626145, 1626149, 1626153, 1626156, 1626157, 1626162, 1626164, 1626166, 1626168, 1626171, 1626172, 1626174, 1626179, 1626192, 1626195, 1626196, 1626220, 1626224, 1627734, 1627736, 1627741, 1627742, 1627745, 1627747, 1627749, 1627751, 1627752, 1627759, 16

In [32]:
'''
from nba_api.stats.endpoints import leaguedashplayerbiostats

# Pull player bio stats for the season
player_bio = leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=season_variable)
player_bio_df = player_bio.get_data_frames()[0]

# Fast lookup: index by PLAYER_ID
bio_lookup = player_bio_df.set_index('PLAYER_ID').to_dict('index')

# Loop through each player and add required bio fields
for player in player_dic:
    player_id = player['id']
    full_name = player['full_name']
    
    if player_id not in bio_lookup:
        raise ValueError(f"Missing bio data for player: {full_name} (ID: {player_id})")
    
    bio = bio_lookup[player_id]
    
    player['Height'] = bio['PLAYER_HEIGHT_INCHES']
    player['Weight'] = bio['PLAYER_WEIGHT']
    player['team_id'] = bio['TEAM_ID']

print(player_dic)
'''

'\nfrom nba_api.stats.endpoints import leaguedashplayerbiostats\n\n# Pull player bio stats for the season\nplayer_bio = leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=season_variable)\nplayer_bio_df = player_bio.get_data_frames()[0]\n\n# Fast lookup: index by PLAYER_ID\nbio_lookup = player_bio_df.set_index(\'PLAYER_ID\').to_dict(\'index\')\n\n# Loop through each player and add required bio fields\nfor player in player_dic:\n    player_id = player[\'id\']\n    full_name = player[\'full_name\']\n    \n    if player_id not in bio_lookup:\n        raise ValueError(f"Missing bio data for player: {full_name} (ID: {player_id})")\n    \n    bio = bio_lookup[player_id]\n    \n    player[\'Height\'] = bio[\'PLAYER_HEIGHT_INCHES\']\n    player[\'Weight\'] = bio[\'PLAYER_WEIGHT\']\n    player[\'team_id\'] = bio[\'TEAM_ID\']\n\nprint(player_dic)\n'

In [33]:

'''
# Importing required library
import pandas as pd

# Define the data for traded players
data = {
    "full_name": [
        "Precious Achiuwa", "Ochai Agbaji", "Marvin Bagley III", "Dalano Banton", 
        "RJ Barrett", "Patrick Beverley", "Bojan Bogdanovic", "Bruce Brown", 
        "Alec Burks", "Spencer Dinwiddie", "Simone Fontecchio", "Daniel Gafford", 
        "Jordan Goodwin", "Quentin Grimes", "Gordon Hayward", "Buddy Hield", 
        "Kyle Lowry", "Chimezie Metu", "Vasilije Micic", "Kelly Olynyk", 
        "Cameron Payne", "Immanuel Quickley", "David Roddy", "Terry Rozier", 
        "Dennis Schroder", "Pascal Siakam", "Xavier Tillman", "P.J. Washington", 
        "Grant Williams"
    ],
    "Player_ID": [
        1630173, 1630534, 1628963, 1630625, 1629628, 201976, 202711, 1628971, 
        202692, 203915, 1631323, 1629655, 1630692, 1629656, 202330, 1627741, 
        200768, 1629002, 203995, 203482, 1626166, 1630193, 1631223, 1626179, 
        203471, 1627783, 1630214, 1629023, 1629684
    ],
    "Team 1": [
        "Raptors", "Jazz", "Pistons", "Celtics", "Knicks", "Sixers", "Pistons", 
        "Pacers", "Pistons", "Nets", "Jazz", "Hornets", "Suns", "Knicks", 
        "Hornets", "Pacers", "Heat", "Suns", "Mavericks", "Jazz", "Heat", 
        "Knicks", "Grizzlies", "Hornets", "Nets", "Raptors", "Grizzlies", 
        "Hornets", "Mavericks"
    ],
    "Team 2": [
        "Knicks", "Raptors", "Wizards", "Blazers", "Raptors", "Bucks", "Knicks", 
        "Raptors", "Knicks", "Raptors", "Pistons", "Mavericks", "Nets", "Pistons", 
        "Thunder", "Sixers", "Sixers", "Grizzlies", "Hornets", "Raptors", "Sixers", 
        "Raptors", "Suns", "Heat", "Raptors", "Pacers", "Celtics", "Mavericks", 
        "Hornets"
    ]
}

# Create the DataFrame
traded_players_df = pd.DataFrame(data)

# Create a correction map for team abbreviations
correction_map = {
    "Blazers": "POR", "Bucks": "MIL", "Celtics": "BOS", "Grizzlies": "MEM", 
    "Heat": "MIA", "Hornets": "CHA", "Jazz": "UTA", "Knicks": "NYK", 
    "Mavericks": "DAL", "Nets": "BKN", "Pacers": "IND", "Pistons": "DET", 
    "Raptors": "TOR", "Sixers": "PHI", "Suns": "PHX", "Thunder": "OKC", 
    "Wizards": "WAS"
}

# Load the player_bio data (assuming it's already in your environment or define the path if needed)
player_bio_df = pd.read_csv('player_bio.csv')

# Extract unique team IDs and their abbreviations to create a mapping
team_id_map = player_bio_df.drop_duplicates(subset=['TEAM_ABBREVIATION'])[['TEAM_ABBREVIATION', 'TEAM_ID']].set_index('TEAM_ABBREVIATION')['TEAM_ID'].to_dict()

# Update the traded_players_df using the correction map and team_id_map
traded_players_df['Team 1'] = traded_players_df['Team 1'].map(correction_map).fillna(traded_players_df['Team 1'])
traded_players_df['Team 2'] = traded_players_df['Team 2'].map(correction_map).fillna(traded_players_df['Team 2'])
traded_players_df['Original_Team_ID'] = traded_players_df['Team 1'].map(team_id_map)
traded_players_df['Traded_Team_ID'] = traded_players_df['Team 2'].map(team_id_map)

# Display the updated DataFrame with corrected team IDs
traded_players_df[['full_name', 'Player_ID', 'Team 1', 'Original_Team_ID', 'Team 2', 'Traded_Team_ID']].to_csv('traded_players.csv', index=False)
'''


'\n# Importing required library\nimport pandas as pd\n\n# Define the data for traded players\ndata = {\n    "full_name": [\n        "Precious Achiuwa", "Ochai Agbaji", "Marvin Bagley III", "Dalano Banton", \n        "RJ Barrett", "Patrick Beverley", "Bojan Bogdanovic", "Bruce Brown", \n        "Alec Burks", "Spencer Dinwiddie", "Simone Fontecchio", "Daniel Gafford", \n        "Jordan Goodwin", "Quentin Grimes", "Gordon Hayward", "Buddy Hield", \n        "Kyle Lowry", "Chimezie Metu", "Vasilije Micic", "Kelly Olynyk", \n        "Cameron Payne", "Immanuel Quickley", "David Roddy", "Terry Rozier", \n        "Dennis Schroder", "Pascal Siakam", "Xavier Tillman", "P.J. Washington", \n        "Grant Williams"\n    ],\n    "Player_ID": [\n        1630173, 1630534, 1628963, 1630625, 1629628, 201976, 202711, 1628971, \n        202692, 203915, 1631323, 1629655, 1630692, 1629656, 202330, 1627741, \n        200768, 1629002, 203995, 203482, 1626166, 1630193, 1631223, 1626179, \n        203471, 16277

In [34]:
from nba_api.stats.endpoints import playerestimatedmetrics

# Pull estimated metrics for all players in the season
players_advance_pct = playerestimatedmetrics.PlayerEstimatedMetrics(season=season_variable)
players_advance_pct_df = players_advance_pct.get_data_frames()[0]

# Create fast lookup dictionary keyed by PLAYER_ID
advance_lookup = players_advance_pct_df.set_index('PLAYER_ID').to_dict('index')

# Add advanced percentage stats to each player in the list
for player in player_dic:
    player_id = player['id']
    full_name = player['full_name']
    
    if player_id not in advance_lookup:
        raise ValueError(f"Missing estimated metrics for player: {full_name} (ID: {player_id})")
    
    metrics = advance_lookup[player_id]
    
    player['AST_PCT'] = metrics['E_AST_RATIO']
    player['DREB_PCT'] = metrics['E_DREB_PCT']
    player['OREB_PCT'] = metrics['E_OREB_PCT']
    player['TOV_PCT'] = metrics['E_TOV_PCT']

print(player_dic)


[{'id': 2544, 'full_name': 'LeBron James', 'first_name': 'LeBron', 'last_name': 'James', 'is_active': True, 'GP_TOTAL': 56, 'MIN_PG': 37.23, 'AST_PCT': 18.2, 'DREB_PCT': 0.21, 'OREB_PCT': 0.034, 'TOV_PCT': 10.241}, {'id': 2546, 'full_name': 'Carmelo Anthony', 'first_name': 'Carmelo', 'last_name': 'Anthony', 'is_active': False, 'GP_TOTAL': 69, 'MIN_PG': 26.09, 'AST_PCT': 7.4, 'DREB_PCT': 0.134, 'OREB_PCT': 0.038, 'TOV_PCT': 6.413}, {'id': 2730, 'full_name': 'Dwight Howard', 'first_name': 'Dwight', 'last_name': 'Howard', 'is_active': False, 'GP_TOTAL': 60, 'MIN_PG': 16.15, 'AST_PCT': 9.6, 'DREB_PCT': 0.271, 'OREB_PCT': 0.135, 'TOV_PCT': 12.868}, {'id': 101108, 'full_name': 'Chris Paul', 'first_name': 'Chris', 'last_name': 'Paul', 'is_active': True, 'GP_TOTAL': 65, 'MIN_PG': 32.92, 'AST_PCT': 41.8, 'DREB_PCT': 0.126, 'OREB_PCT': 0.012, 'TOV_PCT': 9.108}, {'id': 200752, 'full_name': 'Rudy Gay', 'first_name': 'Rudy', 'last_name': 'Gay', 'is_active': False, 'GP_TOTAL': 55, 'MIN_PG': 18.85, '

In [35]:
from nba_api.stats.endpoints import leaguedashplayershotlocations

### Midrange, Paint, Corner, Above the break

players_location_shooting = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(season = season_variable)
players_location_shooting_df = players_location_shooting.get_data_frames()[0]
for player in player_dic:
    player_location_shooting_df = players_location_shooting_df[players_location_shooting_df['']['PLAYER_ID'] == player['id']]
    player['Midrange_volume_pm'] = float(player_location_shooting_df['Mid-Range']['FGA'].iloc[0])/(player['MIN_PG'] * player['GP_TOTAL'])   
    player['Midrange_efficiency'] = float(player_location_shooting_df['Mid-Range']['FG_PCT'].iloc[0])

    player['Paint_volume_pm'] = float(player_location_shooting_df['In The Paint (Non-RA)']['FGA'].iloc[0] + player_location_shooting_df['Restricted Area']['FGA'].iloc[0])/(player['MIN_PG'] * player['GP_TOTAL'])
    player['Paint_efficiency'] = float(player_location_shooting_df['In The Paint (Non-RA)']['FGM'].iloc[0] + player_location_shooting_df['Restricted Area']['FGM'].iloc[0])/float(player_location_shooting_df['In The Paint (Non-RA)']['FGA'].iloc[0] + player_location_shooting_df['Restricted Area']['FGA'].iloc[0])

    player['Corner_volume_pm'] = float(player_location_shooting_df['Corner 3']['FGA'].iloc[0])/(player['MIN_PG'] * player['GP_TOTAL']) 
    player['Corner_efficiency'] = float(player_location_shooting_df['Corner 3']['FG_PCT'].iloc[0])

    player['Above_the_break_volume_pm'] = float(player_location_shooting_df['Above the Break 3']['FGA'].iloc[0])/(player['MIN_PG'] * player['GP_TOTAL'])
    player['Above_the_break_efficiency'] = float(player_location_shooting_df['Above the Break 3']['FG_PCT'].iloc[0])

print(player_dic)

[{'id': 2544, 'full_name': 'LeBron James', 'first_name': 'LeBron', 'last_name': 'James', 'is_active': True, 'GP_TOTAL': 56, 'MIN_PG': 37.23, 'AST_PCT': 18.2, 'DREB_PCT': 0.21, 'OREB_PCT': 0.034, 'TOV_PCT': 10.241, 'Midrange_volume_pm': 0.08825447987414145, 'Midrange_efficiency': 0.424, 'Paint_volume_pm': 0.28251026437972454, 'Paint_efficiency': 0.6808149405772496, 'Corner_volume_pm': 0.017267180844940718, 'Corner_efficiency': 0.306, 'Above_the_break_volume_pm': 0.19761329189209934, 'Above_the_break_efficiency': 0.364}, {'id': 2546, 'full_name': 'Carmelo Anthony', 'first_name': 'Carmelo', 'last_name': 'Anthony', 'is_active': False, 'GP_TOTAL': 69, 'MIN_PG': 26.09, 'AST_PCT': 7.4, 'DREB_PCT': 0.134, 'OREB_PCT': 0.038, 'TOV_PCT': 6.413, 'Midrange_volume_pm': 0.10054382544258725, 'Midrange_efficiency': 0.492, 'Paint_volume_pm': 0.08054615850373012, 'Paint_efficiency': 0.5586206896551724, 'Corner_volume_pm': 0.04166180612261903, 'Corner_efficiency': 0.44, 'Above_the_break_volume_pm': 0.1788

In [36]:
from nba_api.stats.endpoints import leaguedashplayerptshot

#Tight Open Shooting
 
players_contested_shot = leaguedashplayerptshot.LeagueDashPlayerPtShot(season= season_variable, close_def_dist_range_nullable= '0-2 Feet - Very Tight')
players_contested_shot_df_very_tight = players_contested_shot.get_data_frames()[0]
players_contested_shot = leaguedashplayerptshot.LeagueDashPlayerPtShot(season= season_variable, close_def_dist_range_nullable= '2-4 Feet - Tight')
players_contested_shot_df_tight = players_contested_shot.get_data_frames()[0]
players_contested_shot = leaguedashplayerptshot.LeagueDashPlayerPtShot(season= season_variable, close_def_dist_range_nullable= '4-6 Feet - Open')
players_contested_shot_df_open = players_contested_shot.get_data_frames()[0]
players_contested_shot = leaguedashplayerptshot.LeagueDashPlayerPtShot(season= season_variable, close_def_dist_range_nullable= '6+ Feet - Wide Open')
players_contested_shot_df_wide_open = players_contested_shot.get_data_frames()[0]

for player in player_dic:
    player_contested_shot_df_very_tight = players_contested_shot_df_very_tight[players_contested_shot_df_very_tight['PLAYER_ID'] == player['id']]
    player_contested_shot_df_tight = players_contested_shot_df_tight[players_contested_shot_df_tight['PLAYER_ID'] == player['id']]
    player_contested_shot_df_open = players_contested_shot_df_open[players_contested_shot_df_open['PLAYER_ID'] == player['id']]
    player_contested_shot_df_wide_open = players_contested_shot_df_wide_open[players_contested_shot_df_wide_open['PLAYER_ID'] == player['id']]
    
    if player_contested_shot_df_very_tight.empty:
        player_contested_shot_df_very_tight = player_contested_shot_df_tight
    if player_contested_shot_df_wide_open.empty:
        player_contested_shot_df_wide_open = player_contested_shot_df_open

    player['Tight_volume_pm'] = (float(player_contested_shot_df_very_tight['FGA'].iloc[0] + player_contested_shot_df_tight['FGA'].iloc[0]))/(float(player['MIN_PG'] * player['GP_TOTAL']))
    player['Tight_efficiency'] = ((float(player_contested_shot_df_very_tight['FG2M'].iloc[0] + player_contested_shot_df_tight['FG2M'].iloc[0])) + (1.5 * (float(player_contested_shot_df_very_tight['FG3M'].iloc[0] + player_contested_shot_df_tight['FG3M'].iloc[0]))))/(float(player_contested_shot_df_very_tight['FGA'].iloc[0] + player_contested_shot_df_tight['FGA'].iloc[0]))

    player['Open_volume_pm'] = (float(player_contested_shot_df_wide_open['FGA'].iloc[0] + player_contested_shot_df_open['FGA'].iloc[0]))/(float(player['MIN_PG'] * player['GP_TOTAL']))
    player['Open_efficiency'] = ((float(player_contested_shot_df_wide_open['FG2M'].iloc[0] + player_contested_shot_df_open['FG2M'].iloc[0])) + (1.5 * (float(player_contested_shot_df_wide_open['FG3M'].iloc[0] + player_contested_shot_df_open['FG3M'].iloc[0]))))/(float(player_contested_shot_df_wide_open['FGA'].iloc[0] + player_contested_shot_df_open['FGA'].iloc[0]))  


print(player_dic)

[{'id': 2544, 'full_name': 'LeBron James', 'first_name': 'LeBron', 'last_name': 'James', 'is_active': True, 'GP_TOTAL': 56, 'MIN_PG': 37.23, 'AST_PCT': 18.2, 'DREB_PCT': 0.21, 'OREB_PCT': 0.034, 'TOV_PCT': 10.241, 'Midrange_volume_pm': 0.08825447987414145, 'Midrange_efficiency': 0.424, 'Paint_volume_pm': 0.28251026437972454, 'Paint_efficiency': 0.6808149405772496, 'Corner_volume_pm': 0.017267180844940718, 'Corner_efficiency': 0.306, 'Above_the_break_volume_pm': 0.19761329189209934, 'Above_the_break_efficiency': 0.364, 'Tight_volume_pm': 0.256129849199954, 'Tight_efficiency': 0.6451310861423221, 'Open_volume_pm': 0.32951536779095203, 'Open_efficiency': 0.5473071324599709}, {'id': 2546, 'full_name': 'Carmelo Anthony', 'first_name': 'Carmelo', 'last_name': 'Anthony', 'is_active': False, 'GP_TOTAL': 69, 'MIN_PG': 26.09, 'AST_PCT': 7.4, 'DREB_PCT': 0.134, 'OREB_PCT': 0.038, 'TOV_PCT': 6.413, 'Midrange_volume_pm': 0.10054382544258725, 'Midrange_efficiency': 0.492, 'Paint_volume_pm': 0.080546

In [37]:
from nba_api.stats.endpoints import leaguedashplayerptshot

players_pull_up = leaguedashplayerptshot.LeagueDashPlayerPtShot(season=season_variable, general_range_nullable='Pullups')
players_catch_and_shoot = leaguedashplayerptshot.LeagueDashPlayerPtShot(season=season_variable, general_range_nullable='Catch and Shoot')
players_catch_and_shoot_df = players_catch_and_shoot.get_data_frames()[0]
players_pull_up_df = players_pull_up.get_data_frames()[0]

for player in player_dic:
    
    player_catch_and_shoot_df = players_catch_and_shoot_df[players_catch_and_shoot_df['PLAYER_ID'] == player['id']]
    player_pull_up_df = players_pull_up_df[players_pull_up_df['PLAYER_ID'] == player['id']]
    if not player_catch_and_shoot_df.empty and not player_pull_up_df.empty:
        player['Catch_and_Shoot_Volume'] = float(player_catch_and_shoot_df['FG2A'].iloc[0] + player_catch_and_shoot_df['FG3A'].iloc[0])/(player['MIN_PG'] * player['GP_TOTAL'])
        player['Catch_and_Shoot_Efficiency'] = float(player_catch_and_shoot_df['EFG_PCT'].iloc[0])

        player['Pull_up_Volume'] = float(player_pull_up_df['FG2A'].iloc[0] + player_pull_up_df['FG3A'].iloc[0])/(player['MIN_PG'] * player['GP_TOTAL'])
        player['Pull_up_Efficiency'] = float(player_pull_up_df['EFG_PCT'].iloc[0])
    else:
        if player_catch_and_shoot_df.empty:
            player['Catch_and_Shoot_Volume'] = 0
            player['Catch_and_Shoot_Efficiency'] = 0
        if player_pull_up_df.empty:
            player['Pull_up_Volume'] = 0
            player['Pull_up_Efficiency'] = 0
        print(f"Missing data for player: {player['full_name']} (ID: {player['id']})")

print(player_dic)

Missing data for player: Mitchell Robinson (ID: 1629011)
[{'id': 2544, 'full_name': 'LeBron James', 'first_name': 'LeBron', 'last_name': 'James', 'is_active': True, 'GP_TOTAL': 56, 'MIN_PG': 37.23, 'AST_PCT': 18.2, 'DREB_PCT': 0.21, 'OREB_PCT': 0.034, 'TOV_PCT': 10.241, 'Midrange_volume_pm': 0.08825447987414145, 'Midrange_efficiency': 0.424, 'Paint_volume_pm': 0.28251026437972454, 'Paint_efficiency': 0.6808149405772496, 'Corner_volume_pm': 0.017267180844940718, 'Corner_efficiency': 0.306, 'Above_the_break_volume_pm': 0.19761329189209934, 'Above_the_break_efficiency': 0.364, 'Tight_volume_pm': 0.256129849199954, 'Tight_efficiency': 0.6451310861423221, 'Open_volume_pm': 0.32951536779095203, 'Open_efficiency': 0.5473071324599709, 'Catch_and_Shoot_Volume': 0.10120486550784699, 'Catch_and_Shoot_Efficiency': 0.543, 'Pull_up_Volume': 0.1899389892943479, 'Pull_up_Efficiency': 0.468}, {'id': 2546, 'full_name': 'Carmelo Anthony', 'first_name': 'Carmelo', 'last_name': 'Anthony', 'is_active': Fals

In [38]:
'''
from nba_api.stats.endpoints import playerdashptshots
import pandas as pd
import time

# Load the traded players data
traded_players_df = pd.read_csv('traded_players.csv')

for player in player_dic:
    print(player)
    # Check if player is in the traded_players_df
    if int(player['id']) in traded_players_df['Player_ID'].values:
        traded_player = traded_players_df[traded_players_df['Player_ID'] == int(player['id'])]
        team_ids = [traded_player['Original_Team_ID'].iloc[0], traded_player['Traded_Team_ID'].iloc[0]]
        print(player['full_name'], team_ids)
    else:
        team_ids = [player['team_id']]

    # Initialize variables for Catch and Shoot and Pull Ups
    cs_fgm, cs_fga, cs_3pm = 0, 0, 0
    pu_fgm, pu_fga, pu_3pm = 0, 0, 0

    for team_id in team_ids:
        players_shot_clock = playerdashptshots.PlayerDashPtShots(season=season_variable, player_id=player['id'], team_id=team_id)
        players_shot_clock_df = players_shot_clock.get_data_frames()[1].fillna(0)
        
        # Filter for Catch and Shoot and Pull Ups
        cs_df = players_shot_clock_df[players_shot_clock_df['SHOT_TYPE'] == 'Catch and Shoot']
        pu_df = players_shot_clock_df[players_shot_clock_df['SHOT_TYPE'] == 'Pull Ups']

        # Aggregate shooting data for Catch and Shoot
        if not cs_df.empty:
            cs_fgm += cs_df['FGM'].iloc[0]
            cs_fga += cs_df['FGA'].iloc[0]
            cs_3pm += cs_df['FG3M'].iloc[0]

        # Aggregate shooting data for Pull Ups
        if not pu_df.empty:
            pu_fgm += pu_df['FGM'].iloc[0]
            pu_fga += pu_df['FGA'].iloc[0]
            pu_3pm += pu_df['FG3M'].iloc[0]

        time.sleep(0.5)  # Respectful scraping
    # Calculate EFG% for Catch and Shoot
    player['Catch_and_Shoot_Efficiency'] = (cs_fgm + 0.5 * cs_3pm) / cs_fga if cs_fga > 0 else 0
    player['Catch_and_Shoot_Volume'] = (cs_fga) / (player['MIN_PG'] * player['GP_TOTAL'])

    # Calculate EFG% for Pull Ups
    player['Pull_up_Efficiency'] = (pu_fgm + 0.5 * pu_3pm) / pu_fga if pu_fga > 0 else 0
    player['Pull_up_Volume'] = (pu_fga) / (player['MIN_PG'] * player['GP_TOTAL'])

print(player_dic)
'''

"\nfrom nba_api.stats.endpoints import playerdashptshots\nimport pandas as pd\nimport time\n\n# Load the traded players data\ntraded_players_df = pd.read_csv('traded_players.csv')\n\nfor player in player_dic:\n    print(player)\n    # Check if player is in the traded_players_df\n    if int(player['id']) in traded_players_df['Player_ID'].values:\n        traded_player = traded_players_df[traded_players_df['Player_ID'] == int(player['id'])]\n        team_ids = [traded_player['Original_Team_ID'].iloc[0], traded_player['Traded_Team_ID'].iloc[0]]\n        print(player['full_name'], team_ids)\n    else:\n        team_ids = [player['team_id']]\n\n    # Initialize variables for Catch and Shoot and Pull Ups\n    cs_fgm, cs_fga, cs_3pm = 0, 0, 0\n    pu_fgm, pu_fga, pu_3pm = 0, 0, 0\n\n    for team_id in team_ids:\n        players_shot_clock = playerdashptshots.PlayerDashPtShots(season=season_variable, player_id=player['id'], team_id=team_id)\n        players_shot_clock_df = players_shot_clock.g

In [39]:
from nba_api.stats.endpoints import leaguedashptdefend

# Paint Perimeter Defense

players_defending_paint = leaguedashptdefend.LeagueDashPtDefend(season= season_variable, defense_category='Less Than 10Ft')
players_defending_paint_df = players_defending_paint.get_data_frames()[0]
players_defending_perimeter = leaguedashptdefend.LeagueDashPtDefend(season= season_variable, defense_category='Greater Than 15Ft')
players_defending_perimeter_df = players_defending_perimeter.get_data_frames()[0]

for player in player_dic:
    player_defending_paint_df = players_defending_paint_df[players_defending_paint_df['CLOSE_DEF_PERSON_ID'] == player['id']]
    player_defending_perimeter_df = players_defending_perimeter_df[players_defending_perimeter_df['CLOSE_DEF_PERSON_ID'] == player['id']]

    player['Paint_defense_volume_pm'] = float(player_defending_paint_df['FGA_LT_10'].iloc[0])/(player['MIN_PG'] * player['GP_TOTAL'])
    player['Paint_defense_deterance'] = float(player_defending_paint_df['PLUSMINUS'].iloc[0])
    player['Perimeter_defense_volume_pm'] = float(player_defending_perimeter_df['FGA_GT_15'].iloc[0])/(player['MIN_PG'] * player['GP_TOTAL'])
    player['Perimeter_defense_deterance'] = float(player_defending_perimeter_df['PLUSMINUS'].iloc[0])

print(player_dic)


[{'id': 2544, 'full_name': 'LeBron James', 'first_name': 'LeBron', 'last_name': 'James', 'is_active': True, 'GP_TOTAL': 56, 'MIN_PG': 37.23, 'AST_PCT': 18.2, 'DREB_PCT': 0.21, 'OREB_PCT': 0.034, 'TOV_PCT': 10.241, 'Midrange_volume_pm': 0.08825447987414145, 'Midrange_efficiency': 0.424, 'Paint_volume_pm': 0.28251026437972454, 'Paint_efficiency': 0.6808149405772496, 'Corner_volume_pm': 0.017267180844940718, 'Corner_efficiency': 0.306, 'Above_the_break_volume_pm': 0.19761329189209934, 'Above_the_break_efficiency': 0.364, 'Tight_volume_pm': 0.256129849199954, 'Tight_efficiency': 0.6451310861423221, 'Open_volume_pm': 0.32951536779095203, 'Open_efficiency': 0.5473071324599709, 'Catch_and_Shoot_Volume': 0.10120486550784699, 'Catch_and_Shoot_Efficiency': 0.543, 'Pull_up_Volume': 0.1899389892943479, 'Pull_up_Efficiency': 0.468, 'Paint_defense_volume_pm': 0.16163999846513952, 'Paint_defense_deterance': -0.069, 'Perimeter_defense_volume_pm': 0.1563639154291854, 'Perimeter_defense_deterance': -0.0

In [40]:
df = pd.DataFrame.from_dict(player_dic)
df_percentile = pd.DataFrame()

for idx, column in enumerate(df.columns):
    if column in ['team_id', 'id', 'full_name', 'first_name', 'last_name', 'is_active', 'GP_TOTAL', 'MIN_PG', 'team_id', 'Height', 'Weight']:
        df_percentile[column] = df[column]
    else:
        if column in ['Paint_defense_deterance', 'Perimeter_defense_deterance']:
            df_percentile[column + "_percentile"] = df[column].rank(pct=True, ascending=False)
        else:
            df_percentile[column + "_percentile"] = df[column].rank(pct=True)

# Save with season_variable in filename (replacing any slashes with dashes if needed)
df_percentile.to_csv(f'{season_variable}.csv', index=False)
